In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import neuropythy as ny
import ipyvolume as ipv

In [4]:
sub = ny.freesurfer_subject('F:\\ds003812-download\\derivatives\\freesurfer\\wlsubj001')
betas = np.load("F:\\ds003812-download\\derivatives\\processed\\betas\\betas_sub-wlsubj001_ses-04_task-sfprescaled_results.mat.npy")
betas_freq = betas[0]
colores_normalizados = (betas_freq - np.min(betas_freq)) / (np.max(betas_freq) - np.min(betas_freq))
colores_rgba = plt.cm.viridis(betas_freq)

fig = ipv.figure()

ny.cortex_plot(sub.lh, surface='pial', figure=fig, color=colores_rgba[0:sub.lh.vertex_count])
ny.cortex_plot(sub.rh, surface='pial', figure=fig, color=colores_rgba[sub.lh.vertex_count:])

ipv.show()

ny.cortex_plot(sub.lh, surface='inflated', color=colores_rgba[0:sub.lh.vertex_count])


HOLAAAAA desde cortex_plot
HOLAAAAA desde cortex_plot_3d
mesh: [Mesh(<3D>, <240546 faces>, <120275 vertices>)]
mesh len: 1
HOLAAAAAAAAA desde cortex_plot_colors
entro en try
color: [[0.360741 0.785964 0.387814 1.      ]
 [0.127568 0.566949 0.550556 1.      ]
 [0.121831 0.589055 0.545623 1.      ]
 ...
 [0.267004 0.004874 0.329415 1.      ]
 [0.281412 0.155834 0.469201 1.      ]
 [0.267004 0.004874 0.329415 1.      ]]
dentro de if de underlay
HOLAAAA desde apply_cmap
rgba: [[0.360741 0.785964 0.387814 1.      ]
 [0.127568 0.566949 0.550556 1.      ]
 [0.121831 0.589055 0.545623 1.      ]
 ...
 [0.267004 0.004874 0.329415 1.      ]
 [0.281412 0.155834 0.469201 1.      ]
 [0.267004 0.004874 0.329415 1.      ]]
rgba len: 120275
HOLAAAAA desde cortex_plot
HOLAAAAA desde cortex_plot_3d
mesh: [Mesh(<3D>, <241216 faces>, <120610 vertices>)]
mesh len: 1
HOLAAAAAAAAA desde cortex_plot_colors
entro en try
color: [[0.24607  0.73891  0.452024 1.      ]
 [0.134692 0.658636 0.517649 1.      ]
 [0.121

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=0.6…

HOLAAAAA desde cortex_plot
HOLAAAAA desde cortex_plot_3d
mesh: [Mesh(<3D>, <240546 faces>, <120275 vertices>)]
mesh len: 1
HOLAAAAAAAAA desde cortex_plot_colors
entro en try
color: [[0.360741 0.785964 0.387814 1.      ]
 [0.127568 0.566949 0.550556 1.      ]
 [0.121831 0.589055 0.545623 1.      ]
 ...
 [0.267004 0.004874 0.329415 1.      ]
 [0.281412 0.155834 0.469201 1.      ]
 [0.267004 0.004874 0.329415 1.      ]]
dentro de if de underlay
HOLAAAA desde apply_cmap
rgba: [[0.360741 0.785964 0.387814 1.      ]
 [0.127568 0.566949 0.550556 1.      ]
 [0.121831 0.589055 0.545623 1.      ]
 ...
 [0.267004 0.004874 0.329415 1.      ]
 [0.281412 0.155834 0.469201 1.      ]
 [0.267004 0.004874 0.329415 1.      ]]
rgba len: 120275


Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=0.644570721372708, p…

In [23]:
smoothed_betas = np.load("F:\\ds003812-download\\derivatives\\processed\\betas\\smoothed_betas_sub-wlsubj001_ses-04_task-sfprescaled_results.mat.npy")

colores_rgba = plt.cm.viridis(smoothed_betas[0])

fig = ipv.figure()

ny.cortex_plot(sub.lh, surface='pial', figure=fig, color=colores_rgba[0:sub.lh.vertex_count])
ny.cortex_plot(sub.rh, surface='pial', figure=fig, color=colores_rgba[sub.lh.vertex_count:])

ipv.show()

ny.cortex_plot(sub.lh, surface='inflated', color=colores_rgba[0:sub.lh.vertex_count])


HOLAAAAA desde cortex_plot
HOLAAAAA desde cortex_plot_3d
mesh: [Mesh(<3D>, <240546 faces>, <120275 vertices>)]
mesh len: 1
HOLAAAAAAAAA desde cortex_plot_colors
entro en try
color: [[0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 ...
 [0.282884 0.13592  0.453427 1.      ]
 [0.246811 0.283237 0.535941 1.      ]
 [0.165117 0.467423 0.558141 1.      ]]
dentro de if de underlay
HOLAAAA desde apply_cmap
rgba: [[0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 ...
 [0.282884 0.13592  0.453427 1.      ]
 [0.246811 0.283237 0.535941 1.      ]
 [0.165117 0.467423 0.558141 1.      ]]
rgba len: 120275
HOLAAAAA desde cortex_plot
HOLAAAAA desde cortex_plot_3d
mesh: [Mesh(<3D>, <241216 faces>, <120610 vertices>)]
mesh len: 1
HOLAAAAAAAAA desde cortex_plot_colors
entro en try
color: [[0.153894 0.680203 0.504172 1.      ]
 [0.657642 0.860219 0.203082 1.      ]
 [0.993

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=0.6…

HOLAAAAA desde cortex_plot
HOLAAAAA desde cortex_plot_3d
mesh: [Mesh(<3D>, <240546 faces>, <120275 vertices>)]
mesh len: 1
HOLAAAAAAAAA desde cortex_plot_colors
entro en try
color: [[0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 ...
 [0.282884 0.13592  0.453427 1.      ]
 [0.246811 0.283237 0.535941 1.      ]
 [0.165117 0.467423 0.558141 1.      ]]
dentro de if de underlay
HOLAAAA desde apply_cmap
rgba: [[0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 ...
 [0.282884 0.13592  0.453427 1.      ]
 [0.246811 0.283237 0.535941 1.      ]
 [0.165117 0.467423 0.558141 1.      ]]
rgba len: 120275


Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=0.644570721372708, p…